In [1]:
import numpy as np
import pandas as pd
import time
import os
import random
from collections import defaultdict
import deepwalk_graph as graph
from gensim.models.word2vec import Word2Vec

In [2]:
papers = np.load('extr-titles.npy')

In [3]:
conference_list = pd.read_csv('conf_list.csv', header=None, delimiter='\t', names=['abbrev', 'fullname'])

In [4]:
author_conference = defaultdict(lambda: [])

In [5]:
for paper in iter(papers):
    authors = paper[1]
    venue = paper[2]
    for author in iter(authors):
        author_conference[author].append(venue)

noname = author_conference.pop('')

In [6]:
conf_matrix = conference_list['abbrev'].as_matrix()
conf2indx = {k: v for v, k in enumerate(conf_matrix)}
author2indx = {}
conf_link_author = defaultdict(lambda: [])
conf_num = len(conf_matrix)
minimum_publications = 5

In [7]:
f = open('author_conference_net.adjlist', 'w')

In [8]:
counter = 0
for author, confs in author_conference.iteritems():
    if len(confs) >= minimum_publications:
        at_indx = conf_num + counter
        author2indx[author] = at_indx
        line = str(at_indx)
        for cf in iter(confs):
            cf_indx = conf2indx[cf]
            conf_link_author[cf_indx].append(at_indx)
            line += ' ' + str(cf_indx)
        line += '\n'
        f.write(line)
        counter += 1

In [9]:
for cf_indx, at_indxes in conf_link_author.iteritems():
    line = str(cf_indx)
    for at_indx in iter(at_indxes):
        line += ' ' + str(at_indx)
    
    line += '\n'
    f.write(line)

In [10]:
f.close()

In [11]:
G = graph.load_adjacencylist('author_conference_net.adjlist')

In [12]:
st = time.time()
walks = graph.build_deepwalk_corpus(G, num_paths=10, path_length=40, \
                                    alpha=0, rand=random.Random(1234))

print time.time() - st

17.3949999809


In [13]:
# all the node number add "n" prefix
# to solve deepwalk issue #3: TypeError: unsupported operand type(s) for +: 'int' and 'str'
sentences = []
for walk in iter(walks):
    sent = []
    for node in iter(walk):
        sent.append('n' + str(node))
    
    sentences.append(sent)

In [14]:
st = time.time()
model = Word2Vec(sentences, size=100, window=5, min_count=5, workers=4)
print time.time() -st

34.4550001621


In [19]:
indx2author = {k: v for v, k in author2indx.iteritems()}

In [22]:
def author_conf_similarity(model, author2indx, indx2author, conf2indx, author, conf1, conf2, topn=10):
    '''
        return topn similar author by calculating author - conf1 + conf2
    '''
    node1 = 'n' + str(author2indx[author])
    node2 = 'n' + str(conf2indx[conf1])
    node3 = 'n' + str(conf2indx[conf2])
    
    vec1 = model[node1]
    vec2 = model[node2]
    vec3 = model[node3]
    
    results = model.most_similar([vec1 - vec2 + vec3], topn=topn+10)
    counter = 0
    sim_authors = []
    for i, res in enumerate(results):
        index = int(res[0][1:])
        if index > 75: # not conference, but author index
            sim_authors.append((indx2author[index], res[1]))
            counter += 1
            if counter == topn:
                break
    
    return sim_authors

In [24]:
a1 = "Jiawei Han"
c1 = "SIGKDD"
c2 = "SIGIR"
author_conf_similarity(model, author2indx, indx2author, conf2indx, a1, c1, c2)

[('Jamie Callan', 0.6540849208831787),
 ('Jaap Kamps', 0.6511381268501282),
 ('Arjen P. de Vries', 0.6489991545677185),
 ('Norbert Fuhr', 0.6413956880569458),
 ('Tetsuya Sakai', 0.6323436498641968),
 ('Alistair Moffat', 0.6297589540481567),
 ('James Allan', 0.6289938688278198),
 ('Milad Shokouhi', 0.625812292098999),
 ('C. J. van Rijsbergen', 0.625575065612793),
 ('Giridhar Kumaran', 0.6253887414932251)]

In [33]:
author_conference['James Allan']

['SIGIR',
 'IPM',
 'IPM',
 'SIGIR',
 'SIGIR',
 'CIKM',
 'SIGIR',
 'SIGIR',
 'SIGIR',
 'AI',
 'SIGIR',
 'SIGIR',
 'SIGIR',
 'SIGIR',
 'IPM',
 'IPM',
 'ACL',
 'SIGIR',
 'TOIS',
 'SIGIR',
 'SIGIR',
 'AI',
 'CIKM',
 'SIGIR',
 'SIGIR',
 'SIGIR',
 'CIKM',
 'SIGIR',
 'SIGIR',
 'IS',
 'SIGIR',
 'ACL',
 'IS',
 'SIGIR',
 'SIGIR',
 'SIGIR',
 'SIGIR',
 'SIGIR',
 'SIGIR',
 'SIGIR',
 'SIGIR',
 'SIGIR',
 'SIGIR',
 'SIGIR',
 'SIGIR',
 'SIGIR',
 'SIGIR',
 'SIGIR',
 'SIGIR',
 'SIGIR',
 'JASIST',
 'SIGIR',
 'SIGIR',
 'SIGIR',
 'SIGIR',
 'SIGIR',
 'SIGIR',
 'SIGIR',
 'SIGIR']